In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
%%capture
!pip install pyarabic
!pip install camel-tools

In [1]:
import pandas as pd 
import re
import os
import collections
import pyarabic.araby as araby
import pyarabic.number as number

In [2]:
# train data
# data = pd.read_csv("drive/MyDrive/Internship/datasets/Goud/train.csv")

# validation data
# data = pd.read_csv("drive/MyDrive/Internship/datasets/Goud/validation.csv")

# test data
data = pd.read_csv("drive/MyDrive/Internship/datasets/Goud/test.csv")

In [3]:
data

,article,headline,categories
0,لم تمنع ظروف انتشار فيروس كورونا وإقرار الدرا...,روبورطاج.. البياتة بالليل على برا شكل احتجاجي ...,"['آش واقع', 'الرئيسية']"
1,تغطي سماء المغرب، هذه الأيام، سحب كثيفة غريبة...,التخلف باقي فالمجتمع.. البخور وهياكل الموتى و”...,"['الرئيسية', 'جورنالات بلادي']"
2,شهدت برشلونة الإسبانية، السبت، حفلا فنيا لموس...,ف زمان كورونا.. كونصير د الروك تدار ف برشلونة ...,"['الرئيسية', 'كود-تيفي', 'ميديا وثقافة']"
3,تقول عائلة المشتبه فيه بشن هجوم نيس الخميس اب...,معطيات جديدة على الداعشي التونسي للي دبح 3 فكن...,"['آش واقع', 'الرئيسية']"
4,افاد شهود عيان ل”كود” ان وكالة بنكية تابعة للش...,عاجل. ملثمون يهجمون على وكالة بنكية عاود ثاني ...,"['آش واقع', 'الرئيسية']"
...,...,...,...
9492,كود : يونس أفطيط قالت مصادر أمنية ل”كود” أن...,إعتقال مجرم إعتدى على شخص وتسبب له في عاهة مست...,['آش واقع']
9493,قالت تقارير اخبارية، أن “لاگوارديا سيبيل” اعت...,سبتة.. “لاگوارديا سيبيل” شدو 3 ديال الصيادين م...,"['آش واقع', 'الرئيسية']"
9494,علمت “كود” من مصدر مطلع أن عدد من أعضاء الأ...,بنكيران يصارع لتقليص لائحة قيادي الحزب المعتذر...,['آش واقع']
9495,وقفت “كود” خلال جولتها الصحافية ليومه الخميس ،...,صحف. فضيحة تعشير السيارات بالناظور أمام جنايات...,['آش واقع']


In [4]:
data.drop('article', axis=1, inplace=True)

In [ ]:
# pd.set_option('display.max_rows', None)

In [5]:
def str2tuple(categs):
  return tuple(re.sub(r'[\[\]\']', '', categs).split(", "))

In [ ]:
# def stats_categories(data):
#   counts_multi = dict()
#   counts_single = dict()
#   number_rows_multi = 0
#   number_rows_single = 0
#   for index, row in data.iterrows():
#     if isinstance(row["categories"], str):
#       categs = re.sub(r'[\[\]\']', '', row["categories"]).split(", ")
#     else:
#       categs = row["categories"]
#     if len(categs) > 1:
#         number_rows_multi += 1
#         for categ in categs:
#             if categ in counts_multi:
#                 counts_multi[categ] += 1
#             else:
#                 counts_multi[categ] = 1
#     else:
#         number_rows_single += 1
#         if categs[0] in counts_single:
#             counts_single[categs[0]] += 1
#         else:
#             counts_single[categs[0]] = 1

#   print("number of rows containing only one category: ", number_rows_single)

#   print("\n\nfrequency of single categories")
#   for word in counts_single:
#     print(word + " " + str(counts_single[word]))
  
#   print("\n\nnumber of rows containing more than one category: ", number_rows_multi)

#   print("\n\nfrequency of multi categories")
#   for word in counts_multi:
#     print(word + " " + str(counts_multi[word]))

In [6]:
def ignore_categ(categs, ignoring):
  if len(categs) > 1 and ignoring in categs:
    categs_list = list(categs)
    categs_list.remove(ignoring)
    categs = tuple(categs_list)
  return categs

In [7]:
data["categories"] = data["categories"].apply(str2tuple)

In [ ]:
data["categories"].value_counts(sort=True)

(آش واقع, الرئيسية)                         3282
(آش واقع,)                                  2388
(الرئيسية, تبركيك)                           850
(الرئيسية, كود سبور)                         584
(آراء, الرئيسية)                             361
(الرئيسية, كود-تيفي)                         289
(الرئيسية, الزين والحداكة)                   263
(الرئيسية, جورنالات بلادي)                   242
(الرئيسية, ميديا وثقافة)                     221
(ميديا وثقافة,)                              194
(كود سبور,)                                  176
(آراء,)                                      110
(الرئيسية,)                                  101
(تبركيك,)                                     93
(آش واقع, الرئيسية, كود-تيفي)                 88
(الزين والحداكة,)                             38
(آش واقع, ميديا وثقافة)                       24
(جورنالات بلادي,)                             19
(الرئيسية, كود سبور, كود-تيفي)                19
(آش واقع, كود-تيفي)                           11
(آش واقع, تبركيك)   

In [8]:
data["categories"] = data["categories"].apply(ignore_categ, args=("آش واقع",))

In [ ]:
data["categories"].value_counts(sort=True)

(الرئيسية,)                                 3383
(آش واقع,)                                  2388
(الرئيسية, تبركيك)                           857
(الرئيسية, كود سبور)                         591
(الرئيسية, كود-تيفي)                         377
(آراء, الرئيسية)                             371
(الرئيسية, الزين والحداكة)                   264
(الرئيسية, جورنالات بلادي)                   244
(الرئيسية, ميديا وثقافة)                     230
(ميديا وثقافة,)                              218
(كود سبور,)                                  184
(آراء,)                                      111
(تبركيك,)                                    103
(الزين والحداكة,)                             39
(جورنالات بلادي,)                             21
(الرئيسية, كود سبور, كود-تيفي)                19
(كود-تيفي,)                                   19
(آراء, الرئيسية, ميديا وثقافة)                10
(الرئيسية, كود-تيفي, ميديا وثقافة)            10
(الرئيسية, تبركيك, كود سبور)                   9
(الرئيسية, تبركيك, ك

In [9]:
data["categories"] = data["categories"].apply(ignore_categ, args=("الرئيسية",))

In [ ]:
data["categories"].value_counts(sort=True)

(الرئيسية,)                       3383
(آش واقع,)                        2388
(تبركيك,)                          960
(كود سبور,)                        775
(آراء,)                            482
(ميديا وثقافة,)                    448
(كود-تيفي,)                        396
(الزين والحداكة,)                  303
(جورنالات بلادي,)                  265
(كود سبور, كود-تيفي)                20
(تبركيك, كود سبور)                  14
(آراء, ميديا وثقافة)                11
(كود-تيفي, ميديا وثقافة)            10
(تبركيك, كود-تيفي)                   8
(تبركيك, ميديا وثقافة)               7
(الزين والحداكة, كود-تيفي)           5
(كود سبور, ميديا وثقافة)             4
(آراء, كود سبور)                     4
(الزين والحداكة, تبركيك)             3
(جورنالات بلادي, كود سبور)           2
(جورنالات بلادي, ميديا وثقافة)       2
(تبركيك, جورنالات بلادي)             2
(الزين والحداكة, ميديا وثقافة)       1
(جورنالات بلادي, كود-تيفي)           1
(آراء, كود-تيفي, ميديا وثقافة)       1
(الزين والحداكة, كود سبور

In [10]:
data["categories"] = data["categories"].apply(ignore_categ, args=("كود-تيفي",))

In [ ]:
data["categories"].value_counts(sort=True)

(الرئيسية,)                       3383
(آش واقع,)                        2388
(تبركيك,)                          968
(كود سبور,)                        795
(آراء,)                            482
(ميديا وثقافة,)                    458
(كود-تيفي,)                        396
(الزين والحداكة,)                  308
(جورنالات بلادي,)                  266
(تبركيك, كود سبور)                  14
(آراء, ميديا وثقافة)                12
(تبركيك, ميديا وثقافة)               7
(كود سبور, ميديا وثقافة)             4
(آراء, كود سبور)                     4
(الزين والحداكة, تبركيك)             3
(جورنالات بلادي, كود سبور)           2
(جورنالات بلادي, ميديا وثقافة)       2
(تبركيك, جورنالات بلادي)             2
(الزين والحداكة, ميديا وثقافة)       1
(الزين والحداكة, كود سبور)           1
(كود, كود سبور)                      1
Name: categories, dtype: int64

In [11]:
data["categories"] = data["categories"].apply(ignore_categ, args=("تبركيك",))

In [ ]:
data["categories"].value_counts(sort=True)

(الرئيسية,)                       3383
(آش واقع,)                        2388
(تبركيك,)                          968
(كود سبور,)                        809
(آراء,)                            482
(ميديا وثقافة,)                    465
(كود-تيفي,)                        396
(الزين والحداكة,)                  311
(جورنالات بلادي,)                  268
(آراء, ميديا وثقافة)                12
(كود سبور, ميديا وثقافة)             4
(آراء, كود سبور)                     4
(جورنالات بلادي, كود سبور)           2
(جورنالات بلادي, ميديا وثقافة)       2
(الزين والحداكة, ميديا وثقافة)       1
(الزين والحداكة, كود سبور)           1
(كود, كود سبور)                      1
Name: categories, dtype: int64

In [12]:
data["categories"] = data["categories"].apply(ignore_categ, args=("كود سبور",))

In [ ]:
data["categories"].value_counts(sort=True)

(الرئيسية,)                       3383
(آش واقع,)                        2388
(تبركيك,)                          968
(كود سبور,)                        809
(آراء,)                            486
(ميديا وثقافة,)                    469
(كود-تيفي,)                        396
(الزين والحداكة,)                  312
(جورنالات بلادي,)                  270
(آراء, ميديا وثقافة)                12
(جورنالات بلادي, ميديا وثقافة)       2
(الزين والحداكة, ميديا وثقافة)       1
(كود,)                               1
Name: categories, dtype: int64

In [13]:
data.drop(data[data["categories"] == ("آراء", "ميديا وثقافة")].index, inplace=True)
data.drop(data[data["categories"] == ("الزين والحداكة", "ميديا وثقافة")].index, inplace=True)
data.drop(data[data["categories"] == ("جورنالات بلادي", "ميديا وثقافة")].index, inplace=True)
data.drop(data[data["categories"] == ("آراء", "جورنالات بلادي")].index, inplace=True)
data.drop(data[data["categories"] == ("آراء", "الزين والحداكة")].index, inplace=True)
data.drop(data[data["categories"] == ("الزين والحداكة", "جورنالات بلادي")].index, inplace=True)
data.drop(data[data["categories"] == ("كود-تيفي",)].index, inplace=True)
data.drop(data[data["categories"] == ("الرئيسية",)].index, inplace=True)
data.drop(data[data["categories"] == ("كود",)].index, inplace=True)

In [ ]:
data["categories"].value_counts(sort=True)

(آش واقع,)           36892
(تبركيك,)            14404
(كود سبور,)          11442
(آراء,)               7141
(ميديا وثقافة,)       6503
(الزين والحداكة,)     4540
(جورنالات بلادي,)     3980
Name: categories, dtype: int64

In [14]:
# to downsample
happening_df = data[data["categories"] == ("آش واقع",)]

data.drop(data[data["categories"] == ("آش واقع",)].index, inplace=True)

from sklearn.utils import resample

# for train data
# happening_downsampled = resample(happening_df, replace=True, n_samples=14000, random_state=42)

# for validation data
# happening_downsampled = resample(happening_df, replace=True, n_samples=800, random_state=42)

# for test data
happening_downsampled = resample(happening_df, replace=True, n_samples=800, random_state=42)

data = pd.concat([happening_downsampled, data]).sort_index()

In [15]:
data["categories"].value_counts(sort=True)

(تبركيك,)            968
(كود سبور,)          809
(آش واقع,)           800
(آراء,)              486
(ميديا وثقافة,)      469
(الزين والحداكة,)    312
(جورنالات بلادي,)    270
Name: categories, dtype: int64

In [16]:
def remove_punctuation(headlines):
  return re.sub(r'[؟!؛،.:…‼]', ' ', headlines)

In [17]:
data["headline"] = data["headline"].apply(remove_punctuation)

In [ ]:
data.loc[59470:59471]

In [18]:
def remove_special_chars(headlines):
  return re.sub(r'[ـ()_{}–-”“″»«|=#٪>\ufeff\u202A\u202C\u202B\u200E\u200F\u200B]', ' ', headlines)

In [19]:
data["headline"] = data["headline"].apply(remove_special_chars)

In [ ]:
data.loc[59470:59471]

In [20]:
def remove_diacritics(headlines):
  return araby.strip_diacritics(headlines)

In [21]:
data["headline"] = data["headline"].apply(remove_diacritics)

In [22]:
def remove_elongation(headlines):
  return re.sub(r'(.)\1+', r'\1\1', headlines)

In [23]:
data["headline"] = data["headline"].apply(remove_elongation)

In [24]:
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar

def normalize(headlines):
    headlines = normalize_alef_maksura_ar(headlines)
    headlines = normalize_alef_ar(headlines)
    headlines = normalize_teh_marbuta_ar(headlines)
    return headlines

In [25]:
data["headline"] = data["headline"].apply(normalize)

In [26]:
def one_space(headlines):
  return re.sub(r'[ ]+', r' ', headlines)

In [27]:
data["headline"] = data["headline"].apply(one_space)

In [28]:
def remove_tifinagh(headlines):
  return re.sub(r'[ⵜⴰⴳⵍⴷⵉⵜ ⵏ ⵍⵎⵖⵔⵉⴱ]+', r'', headlines)

In [29]:
data["headline"] = data["headline"].apply(remove_tifinagh)

In [30]:
def check_allowed_chars(headlines):
  if re.match(r'^[\u0621-\u064A0-9a-zA-aéÉÇ\u06AF\u067E\u06A4\u06AD ]+', headlines) == None:
    return False
  else:
    return True

In [31]:
data["allowed chars"] = data["headline"].apply(check_allowed_chars)

In [32]:
data["allowed chars"].value_counts()

True    4114
Name: allowed chars, dtype: int64

In [33]:
pd.set_option('display.max_colwidth', None)

In [62]:
data[data["allowed chars"] == False]

,headline,categories,allowed chars


In [34]:
data.drop('allowed chars', axis=1, inplace=True)

In [35]:
# preprocessed train
# data.to_csv(r'drive/MyDrive/Internship/datasets/Goud/train_preprocessed.csv', index=False)

# preprocessed validation
# data.to_csv(r'drive/MyDrive/Internship/datasets/Goud/validation_preprocessed.csv', index=False)

# preprocessed test
data.to_csv(r'drive/MyDrive/Internship/datasets/Goud/test_preprocessed.csv', index=False)

In [48]:
print(re.fullmatch(r'^[\u0621-\u064Aa-zA-a\u06AF\u067E\u06A4\u06AD ]+', "‎​علامات استفهام في الذكري الرابعه ل 20 فبراير	"))

None
